# Objective

Currently, each row has all the stats needed for the team in the 'team' column. However, we don't have the same information for the opponent in the same row. Let's make that happen.

To further clarify, for each row, we need to pull the opposing team's rolling stats, adding them back into the same row. This way, our model will only need one row to make a prediction.

In [1]:
import pandas as pd
import numpy as np
from clean import clean_games

In [2]:
pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 73)

In [3]:
game_df = clean_games("data/games_scraped.pickle")

In [4]:
mask = (game_df.game_id == 'chi-rai-2019-10-06')
subset = game_df.loc[mask]
subset

,team,year,team_year,date,opp,week_num,decade,game_day_of_week,game_outcome,team_record,pts_off,margin,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,game_time,overtime,exp_pts_off,exp_pts_def,exp_pts_st,home,game_id,result_tie,result_win,wins,losses,ties,prev_wins,prev_losses,prev_ties,roll3_ties,roll3_wins,prev_pts_off,prev_pts_def,prev_margin,prev_first_down_off,prev_yards_off,prev_pass_yds_off,prev_rush_yds_off,prev_to_off,prev_first_down_def,prev_yards_def,prev_pass_yds_def,prev_rush_yds_def,prev_to_def,prev_result_tie,prev_result_win,roll3_pts_off,roll3_pts_def,roll3_margin,roll3_first_down_off,roll3_yards_off,roll3_pass_yds_off,roll3_rush_yds_off,roll3_to_off,roll3_yards_def,roll3_pass_yds_def,roll3_rush_yds_def,roll3_to_def,ewma_pts_off,ewma_pts_def,ewma_margin,ewma_first_down_off,ewma_yards_off,ewma_pass_yds_off,ewma_rush_yds_off,ewma_to_off,ewma_yards_def,ewma_pass_yds_def,ewma_rush_yds_def,ewma_to_def
4823,chi,2019,chi-2019,2019-10-06,rai,5,201,Sun,L,3-2,21.0,-3.0,24.0,15.0,236.0,194.0,42.0,2.0,25.0,398.0,229.0,169.0,2.0,1.0,NaN,-3.0,-2.0,-0.0,0,chi-rai-2019-10-06,0,0,3,2,0,3,1,0,3.0,3.0,16.0,6.0,10.0,17.0,269.0,197.0,72.0,0.0,15.0,222.0,182.0,40.0,2.0,0.0,1.0,21.000,11.667,9.333,18.667,280.000,175.000,105.0,0.333,316.667,250.333,66.333,2.667,17.286,11.053,6.233,18.082,274.496,184.250,90.246,0.469,290.353,229.646,60.707,2.148
20606,rai,2019,rai-2019,2019-10-06,chi,5,201,Sun,W,3-2,24.0,3.0,21.0,25.0,398.0,229.0,169.0,2.0,15.0,236.0,194.0,42.0,2.0,1.0,NaN,2.0,3.0,0.0,1,chi-rai-2019-10-06,0,1,3,2,0,2,2,0,1.0,1.0,31.0,24.0,7.0,21.0,377.0,189.0,188.0,1.0,22.0,346.0,265.0,81.0,2.0,0.0,1.0,18.333,28.667,-10.333,19.000,328.667,193.667,135.0,1.333,399.333,291.667,107.667,1.000,20.200,25.924,-5.725,19.070,336.851,207.394,129.456,1.027,384.007,277.499,106.508,0.830


So, in the above table, I need to get all of the `roll3_x` and `ewma_x` columns from the adjacent tables, respectively. We'll prefix these with ``opp_``.

In [5]:
game_df.columns

Index(['team', 'year', 'team_year', 'date', 'opp', 'week_num', 'decade',
       'game_day_of_week', 'game_outcome', 'team_record', 'pts_off', 'margin',
       'pts_def', 'first_down_off', 'yards_off', 'pass_yds_off',
       'rush_yds_off', 'to_off', 'first_down_def', 'yards_def', 'pass_yds_def',
       'rush_yds_def', 'to_def', 'game_time', 'overtime', 'exp_pts_off',
       'exp_pts_def', 'exp_pts_st', 'home', 'game_id', 'result_tie',
       'result_win', 'wins', 'losses', 'ties', 'prev_wins', 'prev_losses',
       'prev_ties', 'roll3_ties', 'roll3_wins', 'prev_pts_off', 'prev_pts_def',
       'prev_margin', 'prev_first_down_off', 'prev_yards_off',
       'prev_pass_yds_off', 'prev_rush_yds_off', 'prev_to_off',
       'prev_first_down_def', 'prev_yards_def', 'prev_pass_yds_def',
       'prev_rush_yds_def', 'prev_to_def', 'prev_result_tie',
       'prev_result_win', 'roll3_pts_off', 'roll3_pts_def', 'roll3_margin',
       'roll3_first_down_off', 'roll3_yards_off', 'roll3_pass_yds_off',


In [6]:
opp_pull_cols = ['game_id', 'team', 'opp', 'prev_wins', 'prev_losses',
       'prev_ties', 'roll3_ties', 'roll3_wins', 'prev_pts_off', 'prev_pts_def',
       'prev_margin', 'prev_first_down_off', 'prev_yards_off',
       'prev_pass_yds_off', 'prev_rush_yds_off', 'prev_to_off',
       'prev_first_down_def', 'prev_yards_def', 'prev_pass_yds_def',
       'prev_rush_yds_def', 'prev_to_def', 'prev_result_tie',
       'prev_result_win', 'roll3_pts_off', 'roll3_pts_def', 'roll3_margin',
       'roll3_first_down_off', 'roll3_yards_off', 'roll3_pass_yds_off',
       'roll3_rush_yds_off', 'roll3_to_off', 'roll3_yards_def',
       'roll3_pass_yds_def', 'roll3_rush_yds_def', 'roll3_to_def',
       'ewma_pts_off', 'ewma_pts_def', 'ewma_margin', 'ewma_first_down_off',
       'ewma_yards_off', 'ewma_pass_yds_off', 'ewma_rush_yds_off',
       'ewma_to_off', 'ewma_yards_def', 'ewma_pass_yds_def',
       'ewma_rush_yds_def', 'ewma_to_def']

In [7]:
# convert all cols to float
for col in opp_pull_cols[3:]:
    game_df[col] = game_df[col].astype(float)

In [8]:
mask = game_df.team_year == 'chi-2019'
game_df[mask][opp_pull_cols]

,game_id,team,opp,prev_wins,prev_losses,prev_ties,roll3_ties,roll3_wins,prev_pts_off,prev_pts_def,prev_margin,prev_first_down_off,prev_yards_off,prev_pass_yds_off,prev_rush_yds_off,prev_to_off,prev_first_down_def,prev_yards_def,prev_pass_yds_def,prev_rush_yds_def,prev_to_def,prev_result_tie,prev_result_win,roll3_pts_off,roll3_pts_def,roll3_margin,roll3_first_down_off,roll3_yards_off,roll3_pass_yds_off,roll3_rush_yds_off,roll3_to_off,roll3_yards_def,roll3_pass_yds_def,roll3_rush_yds_def,roll3_to_def,ewma_pts_off,ewma_pts_def,ewma_margin,ewma_first_down_off,ewma_yards_off,ewma_pass_yds_off,ewma_rush_yds_off,ewma_to_off,ewma_yards_def,ewma_pass_yds_def,ewma_rush_yds_def,ewma_to_def
4822,chi-min-2019-09-29,chi,min,2.0,1.0,0.0,2.0,2.0,31.0,15.0,16.0,21.0,298.0,208.0,90.0,1.0,25.0,356.0,287.0,69.0,5.0,0.0,1.0,16.667,13.000,3.667,18.333,275.000,178.667,96.333,0.667,313.667,245.000,68.667,2.000,17.834,13.205,4.628,18.542,276.836,178.819,98.017,0.668,319.466,249.939,69.527,2.211
4823,chi-rai-2019-10-06,chi,rai,3.0,1.0,0.0,3.0,3.0,16.0,6.0,10.0,17.0,269.0,197.0,72.0,0.0,15.0,222.0,182.0,40.0,2.0,0.0,1.0,21.000,11.667,9.333,18.667,280.000,175.000,105.000,0.333,316.667,250.333,66.333,2.667,17.286,11.053,6.233,18.082,274.496,184.250,90.246,0.469,290.353,229.646,60.707,2.148
4825,chi-nor-2019-10-20,chi,nor,3.0,2.0,0.0,2.0,2.0,21.0,24.0,-3.0,15.0,236.0,194.0,42.0,2.0,25.0,398.0,229.0,169.0,2.0,0.0,0.0,22.667,15.000,7.667,17.667,267.667,199.667,68.000,1.000,325.333,232.667,92.667,3.000,18.225,14.328,3.898,17.302,264.760,186.716,78.044,0.856,317.578,229.482,88.095,2.110
4826,chi-sdg-2019-10-27,chi,sdg,3.0,3.0,0.0,1.0,1.0,25.0,36.0,-11.0,16.0,252.0,235.0,17.0,2.0,24.0,424.0,273.0,151.0,0.0,0.0,0.0,20.667,22.000,-1.333,16.000,252.333,208.667,43.667,1.333,348.000,228.000,120.000,1.333,19.734,19.156,0.579,17.012,261.917,197.472,64.445,1.111,341.286,239.177,102.109,1.640
4827,chi-phi-2019-11-03,chi,phi,3.0,4.0,0.0,0.0,0.0,16.0,17.0,-1.0,26.0,388.0,226.0,162.0,2.0,11.0,231.0,195.0,36.0,1.0,0.0,0.0,20.667,25.667,-5.000,19.000,292.000,218.333,73.667,2.000,351.000,232.333,118.667,1.000,18.982,18.721,0.261,18.824,287.333,203.223,84.110,1.290,319.054,230.272,88.783,1.511
4828,chi-det-2019-11-10,chi,det,3.0,5.0,0.0,0.0,0.0,14.0,22.0,-8.0,10.0,164.0,102.0,62.0,1.0,26.0,373.0,227.0,146.0,0.0,0.0,0.0,18.333,25.000,-6.667,17.333,268.000,187.667,80.333,1.667,342.667,231.667,111.000,0.333,18.055,19.331,-1.276,17.183,264.397,184.398,79.998,1.236,329.087,229.663,99.423,1.230
4829,chi-ram-2019-11-17,chi,ram,4.0,5.0,0.0,1.0,1.0,20.0,13.0,7.0,13.0,226.0,145.0,81.0,0.0,21.0,357.0,259.0,98.0,1.0,0.0,1.0,16.667,17.333,-0.667,16.333,259.333,157.667,101.667,1.000,320.333,227.000,93.333,0.667,18.394,18.229,0.165,16.455,257.713,177.540,80.173,1.021,333.946,234.770,99.176,1.190
4830,chi-nyg-2019-11-24,chi,nyg,4.0,6.0,0.0,1.0,1.0,7.0,17.0,-10.0,17.0,267.0,193.0,74.0,1.0,13.0,283.0,173.0,110.0,2.0,0.0,0.0,13.667,17.333,-3.667,13.333,219.000,146.667,72.333,0.667,337.667,219.667,118.000,1.000,16.516,18.026,-1.510,16.545,259.243,180.087,79.156,1.018,325.551,224.592,100.959,1.324
4831,chi-det-2019-11-28,chi,det,5.0,6.0,0.0,2.0,2.0,19.0,14.0,5.0,20.0,335.0,270.0,65.0,2.0,14.0,243.0,134.0,109.0,1.0,0.0,1.0,15.333,14.667,0.667,16.667,276.000,202.667,73.333,1.000,294.333,188.667,105.667,1.333,16.907,17.393,-0.486,17.088,271.164,194.236,76.928,1.172,312.560,210.336,102.225,1.273
4832,chi-dal-2019-12-05,chi,dal,6.0,6.0,0.0,2.0,2.0,24.0,20.0,4.0,22.0,419.0,331.0,88.0,1.0,19.0,364.0,259.0,105.0,1.0,0.0,1.0,16.667,17.000,-0.333,19.667,340.333,264.667,75.667,1.333,296.667,188.667,108.000,1.333,17.981,17.787,0.193,17.832,293.540,214.936,78.604,1.146,320.346,217.701,102.645,1.231


In [9]:
merged_df = (game_df
             .merge(right=game_df[opp_pull_cols],
                    left_on=['game_id', 'team'],
                    right_on=['game_id', 'opp'],
                    suffixes=[None, '_opp']))



In [10]:
mask = (merged_df.game_id == 'chi-rai-2019-10-06')

In [11]:
merged_df[mask]

,team,year,team_year,date,opp,week_num,decade,game_day_of_week,game_outcome,team_record,pts_off,margin,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,game_time,overtime,exp_pts_off,exp_pts_def,exp_pts_st,home,game_id,result_tie,result_win,wins,losses,ties,prev_wins,prev_losses,prev_ties,roll3_ties,roll3_wins,prev_pts_off,prev_pts_def,prev_margin,prev_first_down_off,prev_yards_off,prev_pass_yds_off,prev_rush_yds_off,prev_to_off,prev_first_down_def,prev_yards_def,...,ewma_yards_def,ewma_pass_yds_def,ewma_rush_yds_def,ewma_to_def,team_opp,opp_opp,prev_wins_opp,prev_losses_opp,prev_ties_opp,roll3_ties_opp,roll3_wins_opp,prev_pts_off_opp,prev_pts_def_opp,prev_margin_opp,prev_first_down_off_opp,prev_yards_off_opp,prev_pass_yds_off_opp,prev_rush_yds_off_opp,prev_to_off_opp,prev_first_down_def_opp,prev_yards_def_opp,prev_pass_yds_def_opp,prev_rush_yds_def_opp,prev_to_def_opp,prev_result_tie_opp,prev_result_win_opp,roll3_pts_off_opp,roll3_pts_def_opp,roll3_margin_opp,roll3_first_down_off_opp,roll3_yards_off_opp,roll3_pass_yds_off_opp,roll3_rush_yds_off_opp,roll3_to_off_opp,roll3_yards_def_opp,roll3_pass_yds_def_opp,roll3_rush_yds_def_opp,roll3_to_def_opp,ewma_pts_off_opp,ewma_pts_def_opp,ewma_margin_opp,ewma_first_down_off_opp,ewma_yards_off_opp,ewma_pass_yds_off_opp,ewma_rush_yds_off_opp,ewma_to_off_opp,ewma_yards_def_opp,ewma_pass_yds_def_opp,ewma_rush_yds_def_opp,ewma_to_def_opp
3686,chi,2019,chi-2019,2019-10-06,rai,5,201,Sun,L,3-2,21.0,-3.0,24.0,15.0,236.0,194.0,42.0,2.0,25.0,398.0,229.0,169.0,2.0,1.0,NaN,-3.0,-2.0,-0.0,0,chi-rai-2019-10-06,0,0,3,2,0,3.0,1.0,0.0,3.0,3.0,16.0,6.0,10.0,17.0,269.0,197.0,72.0,0.0,15.0,222.0,...,290.353,229.646,60.707,2.148,rai,chi,2.0,2.0,0.0,1.0,1.0,31.0,24.0,7.0,21.0,377.0,189.0,188.0,1.0,22.0,346.0,265.0,81.0,2.0,0.0,1.0,18.333,28.667,-10.333,19.000,328.667,193.667,135.0,1.333,399.333,291.667,107.667,1.000,20.200,25.924,-5.725,19.070,336.851,207.394,129.456,1.027,384.007,277.499,106.508,0.830
15771,rai,2019,rai-2019,2019-10-06,chi,5,201,Sun,W,3-2,24.0,3.0,21.0,25.0,398.0,229.0,169.0,2.0,15.0,236.0,194.0,42.0,2.0,1.0,NaN,2.0,3.0,0.0,1,chi-rai-2019-10-06,0,1,3,2,0,2.0,2.0,0.0,1.0,1.0,31.0,24.0,7.0,21.0,377.0,189.0,188.0,1.0,22.0,346.0,...,384.007,277.499,106.508,0.830,chi,rai,3.0,1.0,0.0,3.0,3.0,16.0,6.0,10.0,17.0,269.0,197.0,72.0,0.0,15.0,222.0,182.0,40.0,2.0,0.0,1.0,21.000,11.667,9.333,18.667,280.000,175.000,105.0,0.333,316.667,250.333,66.333,2.667,17.286,11.053,6.233,18.082,274.496,184.250,90.246,0.469,290.353,229.646,60.707,2.148


Awesome!! It worked! But we have duplicates...

Let's get rid of those before we merge them back into the main.

In [12]:
merged_df[mask]

,team,year,team_year,date,opp,week_num,decade,game_day_of_week,game_outcome,team_record,pts_off,margin,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,game_time,overtime,exp_pts_off,exp_pts_def,exp_pts_st,home,game_id,result_tie,result_win,wins,losses,ties,prev_wins,prev_losses,prev_ties,roll3_ties,roll3_wins,prev_pts_off,prev_pts_def,prev_margin,prev_first_down_off,prev_yards_off,prev_pass_yds_off,prev_rush_yds_off,prev_to_off,prev_first_down_def,prev_yards_def,...,ewma_yards_def,ewma_pass_yds_def,ewma_rush_yds_def,ewma_to_def,team_opp,opp_opp,prev_wins_opp,prev_losses_opp,prev_ties_opp,roll3_ties_opp,roll3_wins_opp,prev_pts_off_opp,prev_pts_def_opp,prev_margin_opp,prev_first_down_off_opp,prev_yards_off_opp,prev_pass_yds_off_opp,prev_rush_yds_off_opp,prev_to_off_opp,prev_first_down_def_opp,prev_yards_def_opp,prev_pass_yds_def_opp,prev_rush_yds_def_opp,prev_to_def_opp,prev_result_tie_opp,prev_result_win_opp,roll3_pts_off_opp,roll3_pts_def_opp,roll3_margin_opp,roll3_first_down_off_opp,roll3_yards_off_opp,roll3_pass_yds_off_opp,roll3_rush_yds_off_opp,roll3_to_off_opp,roll3_yards_def_opp,roll3_pass_yds_def_opp,roll3_rush_yds_def_opp,roll3_to_def_opp,ewma_pts_off_opp,ewma_pts_def_opp,ewma_margin_opp,ewma_first_down_off_opp,ewma_yards_off_opp,ewma_pass_yds_off_opp,ewma_rush_yds_off_opp,ewma_to_off_opp,ewma_yards_def_opp,ewma_pass_yds_def_opp,ewma_rush_yds_def_opp,ewma_to_def_opp
3686,chi,2019,chi-2019,2019-10-06,rai,5,201,Sun,L,3-2,21.0,-3.0,24.0,15.0,236.0,194.0,42.0,2.0,25.0,398.0,229.0,169.0,2.0,1.0,NaN,-3.0,-2.0,-0.0,0,chi-rai-2019-10-06,0,0,3,2,0,3.0,1.0,0.0,3.0,3.0,16.0,6.0,10.0,17.0,269.0,197.0,72.0,0.0,15.0,222.0,...,290.353,229.646,60.707,2.148,rai,chi,2.0,2.0,0.0,1.0,1.0,31.0,24.0,7.0,21.0,377.0,189.0,188.0,1.0,22.0,346.0,265.0,81.0,2.0,0.0,1.0,18.333,28.667,-10.333,19.000,328.667,193.667,135.0,1.333,399.333,291.667,107.667,1.000,20.200,25.924,-5.725,19.070,336.851,207.394,129.456,1.027,384.007,277.499,106.508,0.830
15771,rai,2019,rai-2019,2019-10-06,chi,5,201,Sun,W,3-2,24.0,3.0,21.0,25.0,398.0,229.0,169.0,2.0,15.0,236.0,194.0,42.0,2.0,1.0,NaN,2.0,3.0,0.0,1,chi-rai-2019-10-06,0,1,3,2,0,2.0,2.0,0.0,1.0,1.0,31.0,24.0,7.0,21.0,377.0,189.0,188.0,1.0,22.0,346.0,...,384.007,277.499,106.508,0.830,chi,rai,3.0,1.0,0.0,3.0,3.0,16.0,6.0,10.0,17.0,269.0,197.0,72.0,0.0,15.0,222.0,182.0,40.0,2.0,0.0,1.0,21.000,11.667,9.333,18.667,280.000,175.000,105.0,0.333,316.667,250.333,66.333,2.667,17.286,11.053,6.233,18.082,274.496,184.250,90.246,0.469,290.353,229.646,60.707,2.148


Now that we've removed duplicates, we're onto our final step.

Let's merge the `..._y` columns into `game_df`, joining on `game_id` and `team_x`.

To do this, we may need to rename `team` in `game_df` to `team_x`.

Let's zoom back in on our first example to see what's happening.

In [13]:
mask = (game_df.game_id == 'chi-rai-2019-10-06')
game_df[mask]

,team,year,team_year,date,opp,week_num,decade,game_day_of_week,game_outcome,team_record,pts_off,margin,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,game_time,overtime,exp_pts_off,exp_pts_def,exp_pts_st,home,game_id,result_tie,result_win,wins,losses,ties,prev_wins,prev_losses,prev_ties,roll3_ties,roll3_wins,prev_pts_off,prev_pts_def,prev_margin,prev_first_down_off,prev_yards_off,prev_pass_yds_off,prev_rush_yds_off,prev_to_off,prev_first_down_def,prev_yards_def,prev_pass_yds_def,prev_rush_yds_def,prev_to_def,prev_result_tie,prev_result_win,roll3_pts_off,roll3_pts_def,roll3_margin,roll3_first_down_off,roll3_yards_off,roll3_pass_yds_off,roll3_rush_yds_off,roll3_to_off,roll3_yards_def,roll3_pass_yds_def,roll3_rush_yds_def,roll3_to_def,ewma_pts_off,ewma_pts_def,ewma_margin,ewma_first_down_off,ewma_yards_off,ewma_pass_yds_off,ewma_rush_yds_off,ewma_to_off,ewma_yards_def,ewma_pass_yds_def,ewma_rush_yds_def,ewma_to_def
4823,chi,2019,chi-2019,2019-10-06,rai,5,201,Sun,L,3-2,21.0,-3.0,24.0,15.0,236.0,194.0,42.0,2.0,25.0,398.0,229.0,169.0,2.0,1.0,NaN,-3.0,-2.0,-0.0,0,chi-rai-2019-10-06,0,0,3,2,0,3.0,1.0,0.0,3.0,3.0,16.0,6.0,10.0,17.0,269.0,197.0,72.0,0.0,15.0,222.0,182.0,40.0,2.0,0.0,1.0,21.000,11.667,9.333,18.667,280.000,175.000,105.0,0.333,316.667,250.333,66.333,2.667,17.286,11.053,6.233,18.082,274.496,184.250,90.246,0.469,290.353,229.646,60.707,2.148
20606,rai,2019,rai-2019,2019-10-06,chi,5,201,Sun,W,3-2,24.0,3.0,21.0,25.0,398.0,229.0,169.0,2.0,15.0,236.0,194.0,42.0,2.0,1.0,NaN,2.0,3.0,0.0,1,chi-rai-2019-10-06,0,1,3,2,0,2.0,2.0,0.0,1.0,1.0,31.0,24.0,7.0,21.0,377.0,189.0,188.0,1.0,22.0,346.0,265.0,81.0,2.0,0.0,1.0,18.333,28.667,-10.333,19.000,328.667,193.667,135.0,1.333,399.333,291.667,107.667,1.000,20.200,25.924,-5.725,19.070,336.851,207.394,129.456,1.027,384.007,277.499,106.508,0.830


In [14]:
# game_df.rename(columns={'team': 'team_x', 'opp': 'opp_x'}, inplace=True)

In [15]:
# MAY NOT NEED ANY OF THIS

# opp_col_names = ['opp_' + col for col in opp_pull_cols[4:]]
# merged_df_col_names = [col + '_y' for col in opp_pull_cols[4:]]

# game_df[opp_col_names] = (pd.concat([game_df[opp_pull_cols[4:]],
#                                              merged_df[merged_df_col_names]], 
#                                              axis=1,
#                                              join='inner',
#                                              keys=[('game_id', 'opp_x')]))
                                     

In [16]:
mask = (game_df.game_id == 'chi-rai-2019-10-06')
game_df[mask]
#[['game_id', 'week_num', 'team_x', 'opp', 'date', 'prev_yards_off', 'opp_prev_yards_off']]

,team,year,team_year,date,opp,week_num,decade,game_day_of_week,game_outcome,team_record,pts_off,margin,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,game_time,overtime,exp_pts_off,exp_pts_def,exp_pts_st,home,game_id,result_tie,result_win,wins,losses,ties,prev_wins,prev_losses,prev_ties,roll3_ties,roll3_wins,prev_pts_off,prev_pts_def,prev_margin,prev_first_down_off,prev_yards_off,prev_pass_yds_off,prev_rush_yds_off,prev_to_off,prev_first_down_def,prev_yards_def,prev_pass_yds_def,prev_rush_yds_def,prev_to_def,prev_result_tie,prev_result_win,roll3_pts_off,roll3_pts_def,roll3_margin,roll3_first_down_off,roll3_yards_off,roll3_pass_yds_off,roll3_rush_yds_off,roll3_to_off,roll3_yards_def,roll3_pass_yds_def,roll3_rush_yds_def,roll3_to_def,ewma_pts_off,ewma_pts_def,ewma_margin,ewma_first_down_off,ewma_yards_off,ewma_pass_yds_off,ewma_rush_yds_off,ewma_to_off,ewma_yards_def,ewma_pass_yds_def,ewma_rush_yds_def,ewma_to_def
4823,chi,2019,chi-2019,2019-10-06,rai,5,201,Sun,L,3-2,21.0,-3.0,24.0,15.0,236.0,194.0,42.0,2.0,25.0,398.0,229.0,169.0,2.0,1.0,NaN,-3.0,-2.0,-0.0,0,chi-rai-2019-10-06,0,0,3,2,0,3.0,1.0,0.0,3.0,3.0,16.0,6.0,10.0,17.0,269.0,197.0,72.0,0.0,15.0,222.0,182.0,40.0,2.0,0.0,1.0,21.000,11.667,9.333,18.667,280.000,175.000,105.0,0.333,316.667,250.333,66.333,2.667,17.286,11.053,6.233,18.082,274.496,184.250,90.246,0.469,290.353,229.646,60.707,2.148
20606,rai,2019,rai-2019,2019-10-06,chi,5,201,Sun,W,3-2,24.0,3.0,21.0,25.0,398.0,229.0,169.0,2.0,15.0,236.0,194.0,42.0,2.0,1.0,NaN,2.0,3.0,0.0,1,chi-rai-2019-10-06,0,1,3,2,0,2.0,2.0,0.0,1.0,1.0,31.0,24.0,7.0,21.0,377.0,189.0,188.0,1.0,22.0,346.0,265.0,81.0,2.0,0.0,1.0,18.333,28.667,-10.333,19.000,328.667,193.667,135.0,1.333,399.333,291.667,107.667,1.000,20.200,25.924,-5.725,19.070,336.851,207.394,129.456,1.027,384.007,277.499,106.508,0.830
